In [1]:
#This code will get the list of prerequisites needed per class. This code is currently only for electrical eningeering and computer engineering.
#But can most likely be reused in future scrapes. 
from bs4 import BeautifulSoup
import requests
import pandas as pd
from queue import PriorityQueue

In [3]:
#url for the EEC and ECE catalogs and base url

EEC_url = "https://catalog.ucdavis.edu/departments-programs-degrees/electrical-computer-engineering/electrical-engineering-bs/?_gl=1*10xd7u3*_ga*MjgwNTkxMTcxLjE3NDM5NzQ2NTg.*_ga_2N9JP0W2PC*czE3NTE5NTEyNzckbzM0JGcxJHQxNzUxOTUxMzMyJGo1JGwwJGgw#requirementstext"
ECE_url = "https://catalog.ucdavis.edu/departments-programs-degrees/electrical-computer-engineering/computer-engineering-bs/#requirementstext"
base_url = "https://catalog.ucdavis.edu"

In [4]:
#Getting the EEC Info

EEC_page = requests.get(EEC_url)
EEC_soup = BeautifulSoup(EEC_page.text, 'html')


#Getting all the class links
table = EEC_soup.find('table')
EEC_classes = table.find_all('a', class_='bubblelink code')

urls = [base_url + a['href'] for a in EEC_classes if 'href' in a.attrs]
# print(urls)


In [5]:
def get_prereq_links(url):

    prerequisites_names_links = {}

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    first_section = block.find_all('p')

    # print(first_section)

    if len(first_section) == 1:

        return None
    
    elif len(first_section) == 2:

        prereq_description = first_section[1]
        
        if 'href' not in str(prereq_description):

            return None
        
        else:

            prereq_dirty = first_section[1].find_all('a')
            prereq_urls = [base_url + a['href'] for a in prereq_dirty if 'href' in a.attrs]

            # print(prereq_urls)
            
            for url in prereq_urls:

                page = requests.get(url)

                soup = BeautifulSoup(page.text, 'html')

                block = soup.find('div', class_ = 'courseblock')

                titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]

                prerequisites_names_links[titles[0]] = url

          
            




    return prerequisites_names_links





    

    

    

    


In [6]:
full_class_dictionary = {} #This contains every class and link to that class
course_and_prerequisites = {} #This contains every class and their list of prerequisites


class_queue = PriorityQueue()

already_seen = set()

for url in urls:
    class_queue.put(url)


while class_queue.empty() is False:

    url = class_queue.get()

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]

    if titles[0] not in already_seen:
       
       already_seen.add(titles[0])

       print(f'This is the current class: {titles[0]}')

    #    print(f'This is the queue size: {class_queue.qsize()}')

       full_class_dictionary[titles[0]] = url

       course_and_prerequisites[titles[0]] = get_prereq_links(url)

       if course_and_prerequisites[titles[0]] is not None:
            for urs in course_and_prerequisites[titles[0]].values():
                # print(f'Putting: {urs} in queue')
                class_queue.put(urs)



    # print(f'These are the prerequisites: {course_and_prerequisites[titles[0]]}')

    




print(full_class_dictionary)



This is the current class: CHE 002A
This is the current class: CHE 001
This is the current class: CHE 001V
This is the current class: CMN 001
This is the current class: CMN 001V
This is the current class: COM 001
This is the current class: COM 002
This is the current class: COM 003
This is the current class: COM 004
This is the current class: EEC 001
This is the current class: EEC 007
This is the current class: EEC 010
This is the current class: EEC 018
This is the current class: EEC 100
This is the current class: EEC 110A
This is the current class: EEC 111
This is the current class: EEC 130A
This is the current class: EEC 140A
This is the current class: EEC 140AV
This is the current class: EEC 150
This is the current class: EEC 161
This is the current class: EEC 196
This is the current class: ENG 003
This is the current class: ENG 003Y
This is the current class: ENG 006
This is the current class: ENG 017
This is the current class: ENG 017V
This is the current class: ENG 160
This is th

In [14]:
with open('full_class_dictionary.txt', 'w') as file:
    file.write(str(full_class_dictionary))

with open('course_and_prerequistes_dictionary.txt', 'w') as file:
    file.write(str(course_and_prerequisites))

In [15]:
print(len(full_class_dictionary))
print(full_class_dictionary['EEC 150'])
print(course_and_prerequisites['EEC 150'])

103
https://catalog.ucdavis.edu/search/?P=EEC%20150
{'EEC 100': 'https://catalog.ucdavis.edu/search/?P=EEC%20100', 'ENG 006': 'https://catalog.ucdavis.edu/search/?P=ENG%20006', 'MAT 022AL': 'https://catalog.ucdavis.edu/search/?P=MAT%20022AL'}


In [ ]:
#Still needs to be implemented

def get_class_combinations(prereq_string):
    pass
   

In [23]:
def get_total_legacy(course, course_and_prerequisites, visited=None):
    if visited is None:
        visited = set()
    
    
    if course in visited:
        return []

    visited.add(course)

    total_legacy = []

   
    prerequisites = course_and_prerequisites.get(course, [])
    
    for prereq in prerequisites:
        if prereq not in total_legacy:
            total_legacy.append(prereq)
        
      
        deeper_prereqs = get_total_legacy(prereq, course_and_prerequisites, visited)
        for item in deeper_prereqs:
            if item not in total_legacy:
                total_legacy.append(item)

    return total_legacy

    


In [ ]:
df = pd.DataFrame(columns= ['Course', 'Title', 'Units', 'Course Description', 'Prerequisites', 'Min Grade for Prerequisites', 'Number of Prerequisites', 'Total Legacy', 'Learning Activites', 'Credit Limitations', 'Grade Mode', 'General Education'])

#This database doesn't give you the combinations of the possible prerequiste combinations. That still needs to be added. 
#Total legacy just gives you all the classes mentioned for the prerequsites
#Number of prerequisites also just gives all the prerequistes listed (depsite if they are or/and)
#Doesn't work yet


for url in full_class_dictionary.values():

    #print(url)

    page = requests.get(url)

    soup = BeautifulSoup(page.text, 'html')

    block = soup.find('div', class_ = 'courseblock')

    titles = [t.get_text(strip=True).replace('—\xa0', '') for t in block.find_all('b')]
    print(titles[0])

    first_section = block.find_all('p')
    second_section = block.find_all('li')

    #print(second_section)
    #for the prerequisite description, go throug it and: 1) exctract class names 2) see what divides them (and/or). 
    #for max # prerequisites, go through every class needed, and do steps 1 and 2. stop once none is reached. If and, add the class to the max number, if or,
    #add to queue, and get all corresponding classes


    if len(first_section) == 1:

        course_description = first_section[0].get_text(strip=True).replace('Course Description:', '').strip()

        prereq_description = None

        min_grade = None

        num_prererequisites = 0

    elif len(first_section) == 2:


        if 'href' not in str(first_section[1]) or 'strongly' in str(first_section[1]):
            
            course_description = first_section[0].get_text(strip=True).replace('Course Description:', '').strip()

            prereq_description = first_section[1].get_text(strip=True).replace('Prerequisite(s):', '').replace('xa0', " ").strip()

            min_grade = "C-"

            num_prererequisites = len(course_and_prerequisites[titles[0]])
            print(num_prererequisites)

            get_total_legacy = get_total_legacy(titles[0], course_and_prerequisites)
            print(get_total_legacy)


        
        else:

            course_description = first_section[0].get_text(strip=True).replace('Course Description:', '').strip()

            prereq_description = first_section[1].get_text(strip=True).replace('Prerequisite(s):', '').replace('xa0', " ").strip()

            min_grade = "C-"

            num_prererequisites = len(course_and_prerequisites[titles[0]])
            print(num_prererequisites)

            get_total_legacy = get_total_legacy(titles[0], course_and_prerequisites)
            print(get_total_legacy)

            


    learning_activities = second_section[0].get_text(strip=True).replace('Learning Activities:', '').strip()


    if len(second_section) == 3:

        credit_limitations = None


        grade_mode = second_section[1].get_text(strip=True).replace('Grade Mode:', '').strip()


        
        gen_ed = second_section[2].get_text(strip=True).replace('General Education:', '').strip()

    elif len(second_section) == 4:

        credit_limitations = second_section[1].get_text(strip=True).replace('Credit Limitation(s):', '').strip()


        grade_mode = second_section[2].get_text(strip=True).replace('Grade Mode:', '').strip()


        
        gen_ed = second_section[3].get_text(strip=True).replace('General Education:', '').strip()


    #['Course', 'Title', 'Units', 'Course Description', 'Prerequisites', 'Min Grade for Prerequisites', 'Number of Prerequisites', 'Total Legacy', 'Learning Activites', 'Credit Limitations', 'Grade Mode', 'General Education'])
    full_data = titles + [course_description] + [prereq_description] + [min_grade] + [num_prererequisites] + [get_total_legacy] + [learning_activities] + [credit_limitations] + [grade_mode] + [gen_ed]
    print(full_data)

    # length = len(df)
    # df.loc[length] = full_data

CHE 002A
3


TypeError: 'NoneType' object is not iterable

In [ ]:
#Data base should sort the classes from most prerequisite history to least. This only applies to classes, not special cases like "taken highscool math"
#New Data base will contain all the previous data base into, plus # of prerequisites, and the class description of those classes will be added as well. Also the class link will be added